# Лабораторная работа №2

**Требования:**
* Python >= 3.X

Лабораторную работу необходимо выполнять в данном шаблоне. Результатом работы будет являться файл (с измененным именем), который необходимо выложить в Moodle.

**Важно!!!** Имя файлу задавайте по следующему шаблону **lab_2_Группа_ФамилияИО.ipynb**. Например: если Вас зовут Иванов Иван Иванович, и Вы обучаетесь в группе 6207_010302D, то имя файла будет выглядеть так **lab_2_6407_010302D_ИвановИИ.ipynb**.

Необходимо провести исследование различных способов представления документов и их влияние на качество определения тональности.

В качестве входных данных к лабораторной работе взят широко известный набор данных IMDB, содержащий 50K обзоров фильмов ([imdb-dataset-of-50k-movie-reviews](https://disk.yandex.ru/i/DDb0zuyUmts5QA)). Откликами являются значения двух классов positive и negative.

In [8]:
# Код загрузки данных
# Если хотите добавить какие-либо библиотеки
# добавляйте их ИМЕННО ЗДЕСЬ
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


imdb_data = pd.read_csv(r'input/IMDB Dataset.csv')
imdb_data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#### Шаг №1 Подготовка данных

Обязательно предобработайте данные!

In [9]:
# Установите NLTK ресурсы (только при первом запуске)
nltk.download('stopwords')
nltk.download('wordnet')


# Функция для предобработки текста
def preprocess_text(text):
    # Удаление HTML тегов
    text = re.sub(r'<.*?>', ' ', text)
    # Удаление специальных символов и чисел
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Токенизация текста
    words = text.split()
    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Лемматизация слов
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # Соединение слов обратно в строку
    text = ' '.join(words)
    return text


# Применение предобработки ко всему набору данных
imdb_data['review'] = imdb_data['review'].apply(preprocess_text)
imdb_data.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lican\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lican\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,sentiment
0,one reviewer mentioned watching oz episode you...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter matteis love time money visually stunni...,positive


В качестве исследуемых способов представления текстов необходимо рассмотреть:

#### 1.Компоненты вектора: частоты ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).

In [10]:
# Инициализация CountVectorizer
vectorizer = CountVectorizer()

# Векторизация текста
X_counts = vectorizer.fit_transform(imdb_data['review'])

# Вывод результатов векторизации
print("Размер матрицы частот слов:", X_counts.shape)
print("Тип данных матрицы:", type(X_counts))
print("Просмотр содержимого матрицы в формате (документ, термин): частота")
print(X_counts[0])  # Вывод частот для первого документа в наборе данных

Размер матрицы частот слов: (50000, 150902)
Тип данных матрицы: <class 'scipy.sparse._csr.csr_matrix'>
Просмотр содержимого матрицы в формате (документ, термин): частота
  (0, 94282)	1
  (0, 110774)	1
  (0, 82781)	1
  (0, 144631)	2
  (0, 96735)	5
  (0, 41463)	2
  (0, 149894)	1
  (0, 61367)	1
  (0, 111303)	2
  (0, 42753)	1
  (0, 57464)	1
  (0, 47191)	2
  (0, 133283)	1
  (0, 127651)	2
  (0, 17115)	1
  (0, 139807)	1
  (0, 115221)	1
  (0, 143062)	4
  (0, 118246)	1
  (0, 148008)	2
  (0, 53363)	2
  (0, 137422)	1
  (0, 119993)	4
  (0, 44298)	1
  (0, 58567)	1
  :	:
  (0, 146520)	2
  (0, 123242)	1
  (0, 90335)	1
  (0, 65428)	2
  (0, 71130)	1
  (0, 95083)	1
  (0, 52297)	2
  (0, 145342)	1
  (0, 79887)	1
  (0, 83512)	1
  (0, 23681)	1
  (0, 137791)	1
  (0, 13302)	1
  (0, 72883)	1
  (0, 127386)	1
  (0, 121507)	1
  (0, 43404)	1
  (0, 81320)	1
  (0, 11243)	1
  (0, 25367)	1
  (0, 139172)	1
  (0, 142832)	1
  (0, 135859)	1
  (0, 30797)	1
  (0, 120435)	1


#### 2. Компоненты вектора: оценки tf-idf для слова ([TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). 

In [11]:
# Инициализация TfidfVectorizer
vectorizer = TfidfVectorizer()

# Векторизация текста
X_tfidf = vectorizer.fit_transform(imdb_data['review'])

# Вывод результатов векторизации
print("Размер матрицы TF-IDF:", X_tfidf.shape)
print("Тип данных матрицы:", type(X_tfidf))
print("Просмотр содержимого матрицы в формате (документ, термин): TF-IDF вес")
print(X_tfidf[0])  # Вывод весов для первого документа в наборе данных

Размер матрицы TF-IDF: (50000, 150902)
Тип данных матрицы: <class 'scipy.sparse._csr.csr_matrix'>
Просмотр содержимого матрицы в формате (документ, термин): TF-IDF вес
  (0, 120435)	0.04757122457613276
  (0, 30797)	0.07635457258967616
  (0, 135859)	0.05538630507366096
  (0, 142832)	0.1305707580139631
  (0, 139172)	0.07354107464095014
  (0, 25367)	0.07530462963924812
  (0, 11243)	0.046438753026677326
  (0, 81320)	0.038096522346259876
  (0, 43404)	0.047996707643687575
  (0, 121507)	0.060770830794523424
  (0, 127386)	0.05358565572770784
  (0, 72883)	0.046965321075028175
  (0, 13302)	0.08213901875469606
  (0, 137791)	0.051561572140253795
  (0, 23681)	0.05389703531655214
  (0, 83512)	0.053266150286272296
  (0, 79887)	0.09125978188332663
  (0, 145342)	0.026714530645439723
  (0, 52297)	0.04974372271755835
  (0, 95083)	0.05026591237784174
  (0, 71130)	0.046184072375485184
  (0, 65428)	0.1690931491548427
  (0, 90335)	0.10954453998156245
  (0, 123242)	0.0722910362848812
  (0, 146520)	0.194646249

#### 3. Компоненты вектора: частоты N-грам.

In [12]:
# Инициализация CountVectorizer для использования биграмм (2-грамм) и триграмм (3-грамм)
vectorizer = CountVectorizer(ngram_range=(2, 2))

# Векторизация текста
X_ngrams = vectorizer.fit_transform(imdb_data['review'])

# Вывод результатов векторизации
print("Размер матрицы N-грамм:", X_ngrams.shape)
print("Тип данных матрицы:", type(X_ngrams))
print("Пример содержимого матрицы в формате (документ, термин): частота")
print(X_ngrams[0])  # Вывод частот для первого документа в наборе данных

Размер матрицы N-грамм: (50000, 2992968)
Тип данных матрицы: <class 'scipy.sparse._csr.csr_matrix'>
Пример содержимого матрицы в формате (документ, термин): частота
  (0, 1851922)	1
  (0, 2188847)	1
  (0, 1657450)	1
  (0, 2869929)	2
  (0, 1886544)	1
  (0, 829041)	1
  (0, 2982926)	1
  (0, 1245384)	1
  (0, 2198558)	1
  (0, 863684)	1
  (0, 1176393)	1
  (0, 985138)	1
  (0, 2649134)	1
  (0, 2533134)	1
  (0, 1886508)	1
  (0, 330711)	1
  (0, 2774786)	1
  (0, 2270188)	1
  (0, 2832495)	1
  (0, 2337344)	1
  (0, 2200110)	1
  (0, 2939452)	1
  (0, 1106542)	1
  (0, 2736555)	1
  (0, 2370977)	1
  :	:
  (0, 1078751)	1
  (0, 186317)	1
  (0, 2889121)	1
  (0, 1607120)	1
  (0, 1667558)	1
  (0, 453105)	1
  (0, 1327192)	1
  (0, 2746208)	1
  (0, 2039099)	1
  (0, 269948)	1
  (0, 751345)	1
  (0, 1444865)	1
  (0, 2527756)	1
  (0, 2407711)	1
  (0, 2039209)	1
  (0, 883048)	1
  (0, 1886599)	1
  (0, 1631652)	1
  (0, 228069)	1
  (0, 489508)	1
  (0, 2765174)	1
  (0, 2828351)	1
  (0, 1083065)	1
  (0, 2703864)	1
  (0, 6

### Шаг 2. Исследование моделей

<table>
		<tr>
			<td></td>
			<td>$y = 1$</td>
			<td>$y = 0$</td>
		</tr>
		<tr>
			<td>$a(x) = 1$</td>
			<td>True Positive (TP)</td>
			<td>False Positive (FP)</td>
		</tr>
    	<tr>
			<td>$a(x) = 0$</td>
			<td>False Negative (FN)</td>
			<td>True Negative (TN)</td>
		</tr>
</table>

В зависимости от способа представления оценить качество классификации как долю правильных ответов на выборке ($\operatorname{accuracy} = \frac{\operatorname{TP} + \operatorname{TN}}{\operatorname{TP} + \operatorname{TN} + \operatorname{FP} + \operatorname{FN}}$). Используйте перекрестную проверку ([cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)).

Для каждого из нижеперечисленных моделей необходимо определить оптимальные гиперпараметры ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

Качество классификации оцениваем для следующих моделей:

#### 1. Машина опорных векторов ([SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)).

In [13]:
# Инициализация векторизаторов
tfidf_vectorizer = TfidfVectorizer(max_features=500)
count_vectorizer = CountVectorizer(max_features=500)
ngram_vectorizer = CountVectorizer(max_features=500, ngram_range=(2, 2))

# Векторизация данных
X_tfidf = tfidf_vectorizer.fit_transform(imdb_data['review'])
X_counts = count_vectorizer.fit_transform(imdb_data['review'])
X_ngrams = ngram_vectorizer.fit_transform(imdb_data['review'])

y = imdb_data['sentiment']  # Целевая переменная

# Настройка кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Параметры для GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}


# Общая функция для выполнения Grid Search и обучения
def train_and_evaluate(X, y, vectorizer_name):
    print(f"Обучение с использованием {vectorizer_name}")
    model = SVC()
    grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X, y)
    print(f"Лучшие параметры для {vectorizer_name}: {grid_search.best_params_}")
    print(f"Лучшая точность для {vectorizer_name}: {grid_search.best_score_}\n")


# Обучение и оценка для каждой векторизации
train_and_evaluate(X_tfidf, y, "TF-IDF")
train_and_evaluate(X_counts, y, "Count Vectors")
train_and_evaluate(X_ngrams, y, "N-grams")

Обучение с использованием TF-IDF
Лучшие параметры для TF-IDF: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Лучшая точность для TF-IDF: 0.8497

Обучение с использованием Count Vectors
Лучшие параметры для Count Vectors: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Лучшая точность для Count Vectors: 0.84472

Обучение с использованием N-grams
Лучшие параметры для N-grams: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Лучшая точность для N-grams: 0.72146


#### 2. Случайный лес ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)).

In [14]:
# Инициализация векторизаторов
tfidf_vectorizer = TfidfVectorizer(max_features=500)
count_vectorizer = CountVectorizer(max_features=500)
ngram_vectorizer = CountVectorizer(max_features=500, ngram_range=(2, 2))

# Векторизация данных
X_tfidf = tfidf_vectorizer.fit_transform(imdb_data['review'])
X_counts = count_vectorizer.fit_transform(imdb_data['review'])
X_ngrams = ngram_vectorizer.fit_transform(imdb_data['review'])

y = imdb_data['sentiment']  # Целевая переменная

# Настройка кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Параметры для GridSearchCV
param_grid_rf = {
    'n_estimators': [100, 300, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10]
}


# Общая функция для выполнения Grid Search и обучения
def train_and_evaluate_rf(X, y, vectorizer_name):
    print(f"Обучение модели Random Forest с использованием {vectorizer_name}")
    rf_model = RandomForestClassifier(random_state=42)
    grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=kf, scoring='accuracy', n_jobs=-1)
    grid_search_rf.fit(X, y)
    print(f"Лучшие параметры для {vectorizer_name}: {grid_search_rf.best_params_}")
    print(f"Лучшая точность для {vectorizer_name}: {grid_search_rf.best_score_}\n")


# Обучение и оценка для каждой векторизации
train_and_evaluate_rf(X_tfidf, y, "TF-IDF")
train_and_evaluate_rf(X_counts, y, "Count Vectors")
train_and_evaluate_rf(X_ngrams, y, "N-grams")

Обучение модели Random Forest с использованием TF-IDF
Лучшие параметры для TF-IDF: {'max_depth': 20, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 500}
Лучшая точность для TF-IDF: 0.8218799999999999

Обучение модели Random Forest с использованием Count Vectors
Лучшие параметры для Count Vectors: {'max_depth': 20, 'max_features': 'log2', 'min_samples_split': 2, 'n_estimators': 500}
Лучшая точность для Count Vectors: 0.82

Обучение модели Random Forest с использованием N-grams
Лучшие параметры для N-grams: {'max_depth': 20, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 500}
Лучшая точность для N-grams: 0.6975


### Шаг 3. Сравнение результатов

Сравнить точность обученных моделей. Найти наиболее точную.

In [17]:
# Разделение данных на обучающие и тестовые выборки для каждого типа данных
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train_counts, X_test_counts, y_train_counts, y_test_counts = train_test_split(X_counts, y, test_size=0.2, random_state=42)
X_train_ngrams, X_test_ngrams, y_train_ngrams, y_test_ngrams = train_test_split(X_ngrams, y, test_size=0.2, random_state=42)

# Обучение SVC на Count Vectors
svc_counts = SVC(C=10, gamma=0.001, kernel='rbf')
svc_counts.fit(X_train_counts, y_train_counts)
y_pred_counts_svc = svc_counts.predict(X_test_counts)
accuracy_counts_svc = accuracy_score(y_test_counts, y_pred_counts_svc)
print(f"Accuracy of SVC on Count Vectors: {accuracy_counts_svc}")

# Обучение SVC на TF-IDF 
svc_tfidf = SVC(C=1, gamma='scale', kernel='rbf')
svc_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_tfidf = svc_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, y_pred_tfidf)
print(f"Accuracy of SVC on TF-IDF: {accuracy_tfidf}")

# Обучение SVC на N-grams
svc_ngrams = SVC(C=1, gamma=0.1, kernel='rbf')
svc_ngrams.fit(X_train_ngrams, y_train_ngrams)
y_pred_ngrams_svc = svc_ngrams.predict(X_test_ngrams)
accuracy_ngrams_svc = accuracy_score(y_test_ngrams, y_pred_ngrams_svc)
print(f"Accuracy of SVC on N-grams: {accuracy_ngrams_svc}")

# Обучение Random Forest на Count Vectors
rf_counts = RandomForestClassifier(n_estimators=500, max_features='log2', max_depth=20, min_samples_split=2, random_state=42, n_jobs=-1)
rf_counts.fit(X_train_counts, y_train_counts)
y_pred_counts = rf_counts.predict(X_test_counts)
accuracy_counts = accuracy_score(y_test_counts, y_pred_counts)
print(f"Accuracy of Random Forest on Count Vectors: {accuracy_counts}")

# Обучение Random Forest на TF-IDF
rf_tfidf = RandomForestClassifier(n_estimators=500, max_features='log2', max_depth=20, min_samples_split=5, random_state=42, n_jobs=-1)
rf_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_tfidf_rf = rf_tfidf.predict(X_test_tfidf)
accuracy_tfidf_rf = accuracy_score(y_test_tfidf, y_pred_tfidf_rf)
print(f"Accuracy of Random Forest on TF-IDF: {accuracy_tfidf_rf}")

# Обучение Random Forest на N-grams
rf_ngrams = RandomForestClassifier(n_estimators=500, max_features='log2', max_depth=20, min_samples_split=10, random_state=42, n_jobs=-1)
rf_ngrams.fit(X_train_ngrams, y_train_ngrams)
y_pred_ngrams = rf_ngrams.predict(X_test_ngrams)
accuracy_ngrams = accuracy_score(y_test_ngrams, y_pred_ngrams)
print(f"Accuracy of Random Forest on N-grams: {accuracy_ngrams}")

Accuracy of SVC on Count Vectors: 0.8494
Accuracy of SVC on TF-IDF: 0.8524
Accuracy of SVC on N-grams: 0.7208
Accuracy of Random Forest on Count Vectors: 0.8226
Accuracy of Random Forest on TF-IDF: 0.8246
Accuracy of Random Forest on N-grams: 0.7011
